# Common imports & library functions

In [1]:
import doctest

# Day 1: Calorie Counting

In [21]:
def parse_calories(calories_txt):
  r"""
  >>> parse_calories("1\n2\n3\n\n4\n5")
  [[1, 2, 3], [4, 5]]
  """
  return [[int(c) for c in items_per_elf.split('\n')]
          for items_per_elf in calories_txt.split('\n\n')]

def max_calories(calories, n=1):
  """
  >>> max_calories([[1, 2, 3], [4, 5]], n=1)
  9
  >>> max_calories([[1, 2, 3], [4, 5]], n=2)
  [9, 6]
  >>> max_calories([[1000, 2000, 3000], [4000], [5000, 6000], [7000, 8000, 9000], [10000]])
  24000
  """
  sums = (sum(cs) for cs in calories)
  if n == 1:
    return max(sums)
  else:
    return sorted(sums, reverse=True)[:n]

In [22]:
doctest.run_docstring_examples(parse_calories, globs=None, verbose=True)
doctest.run_docstring_examples(max_calories, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_calories("1\n2\n3\n\n4\n5")
Expecting:
    [[1, 2, 3], [4, 5]]
ok
Finding tests in NoName
Trying:
    max_calories([[1, 2, 3], [4, 5]], n=1)
Expecting:
    9
ok
Trying:
    max_calories([[1, 2, 3], [4, 5]], n=2)
Expecting:
    [9, 6]
ok
Trying:
    max_calories([[1000, 2000, 3000], [4000], [5000, 6000], [7000, 8000, 9000], [10000]])
Expecting:
    24000
ok


In [23]:
# Final answers
with open('day1.txt') as f:
    calories = parse_calories(f.read().strip())
    print('Part 1: ', max_calories(calories))
    print('Part 2: ', sum(max_calories(calories, 3)))

Part 1:  71506
Part 2:  209603


# Day 2: Rock Paper Scissors

In [31]:
SHAPE_SCORES = {'A': 1, 'B': 2, 'C': 3}
RESPONSES = {'X': 'A', 'Y': 'B', 'Z': 'C'}
BEATS = {('A', 'B'), ('B', 'C'), ('C', 'A')}

def bad_response(them, you):
  return RESPONSES[you]

def smart_response(them, you):
  if you == 'Y':    # tie
    return them
  elif you == 'X':  # lose
    return next(l for (l, w) in BEATS if them == w)
  elif you == 'Z':  # win
    return next(w for (l, w) in BEATS if them == l)

def shape_score(shape):
  return SHAPE_SCORES[shape]

def outcome_score(them, you):
  if them == you:  # tie
    return 3
  elif (them, you) in BEATS:  # win
    return 6
  else:  # lose
    return 0

def score_game(game, response_fn):
  them, you = game.split(' ')
  you = response_fn(them, you)
  return shape_score(you) + outcome_score(them, you)

def score_strategy(strategy, response_fn):
  r"""
  >>> score_strategy("A Y\nB X\nC Z", response_fn=bad_response)
  15
  >>> score_strategy("A Y\nB X\nC Z", response_fn=smart_response)
  12
  """
  return sum(score_game(g, response_fn) for g in strategy.split('\n'))


In [32]:
doctest.run_docstring_examples(score_strategy, globs=None, verbose=True)

Finding tests in NoName
Trying:
    score_strategy("A Y\nB X\nC Z", response_fn=bad_response)
Expecting:
    15
ok
Trying:
    score_strategy("A Y\nB X\nC Z", response_fn=smart_response)
Expecting:
    12
ok


In [33]:
# Final answers
with open('day2.txt') as f:
    strategy = f.read().strip()
    print('Part 1: ', score_strategy(strategy, bad_response))
    print('Part 2: ', score_strategy(strategy, smart_response))

Part 1:  11906
Part 2:  11186


# Day 3: Rucksack Reorganization

In [80]:
from dataclasses import dataclass
import string
from typing import List
from functools import reduce

PRIORITIES = {}

for c in string.ascii_lowercase:
  PRIORITIES[c] = ord(c) - 96
for C in string.ascii_uppercase:
    PRIORITIES[C] = ord(C) - 38

def priority(c):
  """
  >>> priority('a')
  1
  >>> priority('Z')
  52
  """
  return PRIORITIES[c]

@dataclass
class Rucksack:
  contents: str
  
  @property
  def c1(self):
    return self.contents[:len(self.contents)//2]

  @property
  def c2(self):
    return self.contents[len(self.contents)//2:]
  
  def __iter__(self):
    return iter(self.contents)

  def shared(self):
    """
    >>> Rucksack("vJrwpWtwJgWrhcsFMMfFFhFp").shared()
    'p'
    """
    return next(iter(set(self.c1) & set(self.c2)))

@dataclass
class Group:
  rucksacks: List[Rucksack]
  
  def shared(self):
    """
    >>> Group([Rucksack(r) for r in '''vJrwpWtwJgWrhcsFMMfFFhFp
    ... jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
    ... PmmdzqPrVvPwwTWBwg'''.splitlines()]).shared()
    'r'
    """
    common = reduce(lambda a, b: a & b, (set(r) for r in self.rucksacks))
    return next(iter(common))

  @staticmethod
  def from_rucksacks(rucksacks):
    return [Group(g) for g in zip(*([iter(rucksacks)] * 3))]

def shared_priority(objs):
  r"""
  >>> rucksacks = [Rucksack(r) for r in '''vJrwpWtwJgWrhcsFMMfFFhFp
  ... jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
  ... PmmdzqPrVvPwwTWBwg
  ... wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
  ... ttgJtRGJQctTZtZT
  ... CrZsJsPPZsGzwwsLwLmpwMDw'''.splitlines()]
  >>> shared_priority(rucksacks)
  157
  >>> shared_priority(Group.from_rucksacks(rucksacks))
  70
  """
  return sum(priority(o.shared()) for o in objs)
    

In [83]:
doctest.run_docstring_examples(priority, globs=None, verbose=False)
doctest.run_docstring_examples(shared_priority, globs=None, verbose=False)
doctest.run_docstring_examples(Rucksack.shared, globs=None, verbose=False)
doctest.run_docstring_examples(Group.shared, globs=None, verbose=False)

In [87]:
# Final answers
with open('day3.txt') as f:
    rucksacks = [Rucksack(cs.strip()) for cs in f]
    print('Part 1: ', shared_priority(rucksacks))
    print('Part 2: ', shared_priority(Group.from_rucksacks(rucksacks)))

Part 1:  7766
Part 2:  2415


# Day 4: Camp Cleanup

In [81]:
from dataclasses import dataclass

@dataclass
class Assignment:
  start: int
  end: int

  def __repr__(self) -> str:
    return f'Assignment({self.start}, {self.end})'

  def __contains__(self, other):
    """
    >>> Assignment(6, 6) in Assignment(4, 6)
    True
    >>> 5 in Assignment(4, 6)
    True
    """
    return self.contains(other)
  
  def contains(self, other):
    """
    >>> Assignment(5, 7).contains(Assignment(7, 9))
    False
    >>> Assignment(2, 8).contains(Assignment(3, 7))
    True
    >>> Assignment(4, 6).contains(Assignment(6, 6))
    True
    >>> Assignment(4, 6).contains(1)
    False
    >>> Assignment(4, 6).contains(5)
    True
    """
    if isinstance(other, Assignment):
      return (other.start >= self.start) and (other.end <= self.end)
    else:
      return self.start <= other <= self.end
    
  def overlaps(self, other):
    """
    >>> Assignment(5, 7).overlaps(Assignment(7, 9))
    True
    >>> Assignment(7, 7).overlaps(Assignment(5, 7))
    True
    >>> Assignment(7, 12).overlaps(Assignment(4, 9))
    True
    >>> Assignment(2, 3).overlaps(Assignment(4, 5))
    False
    """
    return (self.contains(other.start) or self.contains(other.end)
            or self in other or other in self)

  @staticmethod
  def parse(range):
    """
    >>> Assignment.parse("2-4")
    Assignment(2, 4)
    >>> Assignment.parse("1-19")
    Assignment(1, 19)
    """
    return Assignment(*(int(s) for s in range.split('-')))

def parse_line(l):
  """
  >>> parse_line("2-4,6-8")
  [Assignment(2, 4), Assignment(6, 8)]
  """
  return [Assignment.parse(r) for r in l.strip().split(',')]

def num_redundant_pairs(pairs):
  r"""
  >>> num_redundant_pairs(TEST_CASE)
  2
  """
  return sum(a in b or b in a for (a, b) in pairs)

def num_overlapping_pairs(pairs):
  r"""
  >>> num_overlapping_pairs(TEST_CASE)
  4
  """
  return sum(a.overlaps(b) for (a, b) in pairs)

In [80]:
TEST_CASE =  [parse_line(l) for l in '''2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8'''.splitlines()]
doctest.testmod(verbose=False, extraglobs={'TEST_CASE':TEST_CASE})

TestResults(failed=0, attempted=18)

In [82]:
# Final answers
with open('day4.txt') as f:
    pairs = [parse_line(l) for l in f]
    print('Part 1: ', num_redundant_pairs(pairs))
    print('Part 2: ', num_overlapping_pairs(pairs))

Part 1:  556
Part 2:  876


# Day 5: Supply Stacks

In [14]:
import re
from dataclasses import dataclass

def is_empty(cell):
  """
  >>> is_empty(' ')
  True
  >>> is_empty('X')
  False
  """
  return cell == ' '

def parse_stacks(drawing):
  """
  >>> parse_stacks('''    [D]    
  ... [N] [C]    
  ... [Z] [M] [P]
  ...  1   2   3 ''')
  [['Z', 'N'], ['M', 'C', 'D'], ['P']]
  """
  layers = drawing.splitlines()[-2::-1]
  return [[l[col] for l in layers if not is_empty(l[col])]
          for col in range(1, len(layers[0]), 4)]

@dataclass
class Move:
  num: int
  src: int
  tgt: int

_move_re = re.compile('move (\d+) from (\d+) to (\d+)')
def parse_moves(lines):
  """
  >>> parse_moves('''move 1 from 2 to 1
  ... move 3 from 1 to 3''')
  [Move(num=1, src=1, tgt=0), Move(num=3, src=0, tgt=2)]
  """
  def make_move(line):
    num, src, tgt = _move_re.match(line).groups()
    return Move(int(num), int(src)-1, int(tgt)-1)
  return [make_move(line) for line in lines.splitlines()]

def parse_crate_plan(plan):
  """
  >>> stacks, moves = parse_crate_plan(TEST_PLAN)
  >>> len(stacks) == 3
  True
  >>> len(moves) == 4
  True
  """
  stacks, moves = plan.split('\n\n')
  return parse_stacks(stacks), parse_moves(moves)

def apply_moves_9000(stacks, moves):
  """
  >>> stacks, moves = parse_crate_plan(TEST_PLAN)
  >>> apply_moves_9000(stacks, moves)
  [['C'], ['M'], ['P', 'D', 'N', 'Z']]
  """
  stacks = [s[:] for s in stacks]
  for m in moves:
    for _ in range(m.num):
      stacks[m.tgt].append(stacks[m.src].pop())
  return stacks

def apply_moves_9001(stacks, moves):
  """
  >>> stacks, moves = parse_crate_plan(TEST_PLAN)
  >>> apply_moves_9001(stacks, moves)
  [['M'], ['C'], ['P', 'Z', 'N', 'D']]
  """
  stacks = [s[:] for s in stacks]
  for m in moves:
    stacks[m.tgt].extend(stacks[m.src][-m.num:])
    stacks[m.src] = stacks[m.src][:-m.num]
  return stacks

def top_layer(stacks):
  return ''.join(s[-1] for s in stacks)

In [15]:
import doctest

TEST_PLAN = '''    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2'''

doctest.testmod(verbose=False, extraglobs={'TEST_PLAN':TEST_PLAN})

TestResults(failed=0, attempted=11)

In [16]:
# Final answers
with open('day5.txt') as f:
    stacks, moves = parse_crate_plan(f.read())
    print('Part 1: ', top_layer(apply_moves_9000(stacks, moves)))
    print('Part 2: ', top_layer(apply_moves_9001(stacks, moves)))

Part 1:  ZBDRNPMVH
Part 2:  WDLPFNNNB
